In [15]:
import pandas as pd
import numpy as np
import utils as ut
from DeepPurpose import utils, dataset

In [2]:
# df_bindingdb = pd.read_csv('../data/BindingDB_All_202407.tsv', sep='\t', on_bad_lines='skip')
# df_bindingdb.shape

In [3]:
# print(list(df_bindingdb.columns))

In [4]:
# Preprocess bindingdb dataset
# X_smiles, X_names, X_targets, y = dataset.process_BindingDB('../data/BindingDB_All_202407.tsv')
# print('Drug 1: ' + X_names[0])
# print('Target 1: ' + X_targets[0])
# print('Score 1: ' + str(y[0]))
df_bindingdb = dataset.process_BindingDB(path='../data/BindingDB_All_202407.tsv', y='Kd')
df_bindingdb.shape

Loading Dataset from path...


c:\Users\debna\anaconda3\envs\gramseq\lib\site-packages\DeepPurpose\dataset.py:205: DtypeWarning: Columns (8,9,10,11,12,13,15,17,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,146,147,148,149,150,151,158,159,160,161,162,163,170,172,173,174,175) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep = '\t', on_bad_lines='skip')


Beginning Processing...
There are 139 drug target pairs.
Default set to logspace (nM -> p) for easier regression


(139, 8)

In [5]:
df_bindingdb = dataset.process_BindingDB(path='../data/BindingDB_All_202407.tsv', y='IC50')
df_bindingdb.shape

Loading Dataset from path...


c:\Users\debna\anaconda3\envs\gramseq\lib\site-packages\DeepPurpose\dataset.py:205: DtypeWarning: Columns (8,9,10,11,12,13,15,17,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,146,147,148,149,150,151,158,159,160,161,162,163,170,172,173,174,175) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep = '\t', on_bad_lines='skip')


Beginning Processing...
There are 1724960 drug target pairs.
Default set to logspace (nM -> p) for easier regression


(1724960, 8)

In [24]:
# smiles_list = ['CCNC(=O)CCC(N)C(O)=O', 'NC(CCCNC(N)=O)C(O)=O', 'CCCN(CCC)C1CCc2ccc(O)cc2C1']
drugs_list = df_bindingdb['Name']
set(drugs_list)

{'CHEMBL1822297',
 'US11286268, Compound 1585',
 'US10550091, No. LC-54::US10947203, No. LC-54',
 'US10323037, Example 118::US9951077, 118',
 'US11214565, Example D-148',
 'US9206199, 131',
 'US9212182, 755',
 'CHEMBL4172159',
 '(S)-1-(3-(5-chloro-4-oxo-2-(1-(thiazolo[5,4-d]pyrimidin-7-yloxy)ethyl)quinazolin-3(4H)-yl)phenyl)-3-(2,2-difluoroethyl)urea::US10221197, Compound 148',
 'CHEMBL3914503',
 'US9085555, 177',
 'CHEMBL4572598',
 'US9120791, Example 83::US9603854, 83',
 'bisarylimidazole derivative, 1::methyl 2-[4-(4,5-diphenyl-1H-imidazol-1-yl)phenoxy]acetate',
 'US9035059, 18-15',
 '4-(((R)-2-cyano-1- phenylethyl)amino)-6-(((S)-(1- cyclopropyl-1H-1,2,3-triazol-4- yl)(6-fluoropyridin-3-yl)methyl- d)amino)quinoline-3,8-dicarbonitrile::US11066414, Compound 100',
 'US11629146, Compound 1',
 'CHEMBL2323405',
 'CHEMBL3797559',
 'CHEMBL5076986',
 "(1R,2S)-2-{3-[(2,5-dimethoxypyridin-3-yl)amino]-1H-indazol-6-yl}-5'-methoxyspiro[cyclopropane-1,3'-indol]-2'(1'H)-one::US20230365537, Example 

In [9]:
# df_cinfo = pd.read_csv('../data/compoundinfo_beta.csv')
# df_cinfo.head()

In [10]:
# smiles_cmap = {}
# for cmap, smiles_q, smiles_cinfo in zip(df_cinfo['cmap_name'], smiles_list, df_cinfo['canonical_smiles']):
#     if smiles_q == smiles_cinfo:
#         smiles_cmap[smiles_q] = cmap

# print(smiles_cmap)

In [11]:
df_l1000 = pd.read_csv('../data/l1000_cp_10uM_all.csv')
df_l1000.head()

,0,2,3,4,5,6,7,8,9,10,...,242,243,244,245,246,247,248,249,250,251
0,ABY001_A375_XH_A13_afatinib_10uM up,RRAGC,VPS8,KCNJ2,DUSP5,BEX1,LIMS1,RAB13,COLEC12,ARMCX2,...,GEM,BEX3,VEGFA,NUPR1,TSPAN6,CA12,TMEM158,CHI3L1,CDC20,HMOX1
1,ABY001_A375_XH_A13_afatinib_10uM down,PCNA,S100A7,DNMT1,S100P,S100A9,PUF60,TMEM45A,GOLT1B,PRSS23,...,AKAP12,SCEL,CTSC,EAF2,MORF4L1,KCNK3,MYB,MAF,LTF,MFNG
2,ABY001_A375_XH_A14_erlotinib_10uM up,CDKN2C,TIMP3,COLGALT2,AMPD3,TGFB1,SERPINB3,MMP7,PIGR,WDR61,...,RHOBTB3,TMSB15A,RPS4Y1,JCHAIN,PLAT,FAM129A,ASNS,PCNA,TNFSF10,CHAC1
3,ABY001_A375_XH_A14_erlotinib_10uM down,PCP4,SPINK1,STEAP1,HOXC6,ITGB1BP1,MRPS16,XIST,UCHL1,FABP4,...,PEG3,WBP1L,SCG5,ATP5F1E,CCL19,EGLN1,MAST4,ATP6V1H,GPX2,EBP
4,ABY001_A375_XH_A15_neratinib_10uM up,ITGAE,PRPF4,UBE2S,OGT,RRAGC,ASNA1,TNFRSF1A,FGL2,GOLGA8A,...,HSPA1A,CHI3L1,KLK11,MSMO1,ADM,HSPA8,TMEM158,KRT14,HMGCS1,HMOX1


In [12]:

# Extract up genes
df_up = df_l1000[df_l1000['0'].str.contains('up')]
df_up.head()

,0,2,3,4,5,6,7,8,9,10,...,242,243,244,245,246,247,248,249,250,251
0,ABY001_A375_XH_A13_afatinib_10uM up,RRAGC,VPS8,KCNJ2,DUSP5,BEX1,LIMS1,RAB13,COLEC12,ARMCX2,...,GEM,BEX3,VEGFA,NUPR1,TSPAN6,CA12,TMEM158,CHI3L1,CDC20,HMOX1
2,ABY001_A375_XH_A14_erlotinib_10uM up,CDKN2C,TIMP3,COLGALT2,AMPD3,TGFB1,SERPINB3,MMP7,PIGR,WDR61,...,RHOBTB3,TMSB15A,RPS4Y1,JCHAIN,PLAT,FAM129A,ASNS,PCNA,TNFSF10,CHAC1
4,ABY001_A375_XH_A15_neratinib_10uM up,ITGAE,PRPF4,UBE2S,OGT,RRAGC,ASNA1,TNFRSF1A,FGL2,GOLGA8A,...,HSPA1A,CHI3L1,KLK11,MSMO1,ADM,HSPA8,TMEM158,KRT14,HMGCS1,HMOX1
6,ABY001_A375_XH_A16_lapatinib_10uM up,SLC1A3,TPPP,SELENOP,ADGRL3,SERPINA4,MAP2K5,MMP26,NLRP2,RHOA,...,NNT,CRIP1,TGM2,IL32,MYRF,SPRY1,SLC11A2,ALDH1A1,FGFR3,CASP10
8,ABY001_A375_XH_D13_pazopanib_10uM up,TMEM185B,COL9A3,DDX42,UPK1B,SCG2,LARP6,PRKCQ,SCN3A,CD74,...,KDM5D,NEBL,XBP1,ALDH1A1,CIAO3,GOLT1B,ALDH3A1,AKR1B10,ZFP36,RPS4Y1


In [16]:
# Clean the drug names in the replicates - up
df_up['0'] = df_up['0'].apply(ut.extract_drug_name)
df_up.head()

C:\Users\debna\AppData\Local\Temp\ipykernel_57364\3558858950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_up['0'] = df_up['0'].apply(ut.extract_drug_name)


,0,2,3,4,5,6,7,8,9,10,...,242,243,244,245,246,247,248,249,250,251
0,afatinib,RRAGC,VPS8,KCNJ2,DUSP5,BEX1,LIMS1,RAB13,COLEC12,ARMCX2,...,GEM,BEX3,VEGFA,NUPR1,TSPAN6,CA12,TMEM158,CHI3L1,CDC20,HMOX1
2,erlotinib,CDKN2C,TIMP3,COLGALT2,AMPD3,TGFB1,SERPINB3,MMP7,PIGR,WDR61,...,RHOBTB3,TMSB15A,RPS4Y1,JCHAIN,PLAT,FAM129A,ASNS,PCNA,TNFSF10,CHAC1
4,neratinib,ITGAE,PRPF4,UBE2S,OGT,RRAGC,ASNA1,TNFRSF1A,FGL2,GOLGA8A,...,HSPA1A,CHI3L1,KLK11,MSMO1,ADM,HSPA8,TMEM158,KRT14,HMGCS1,HMOX1
6,lapatinib,SLC1A3,TPPP,SELENOP,ADGRL3,SERPINA4,MAP2K5,MMP26,NLRP2,RHOA,...,NNT,CRIP1,TGM2,IL32,MYRF,SPRY1,SLC11A2,ALDH1A1,FGFR3,CASP10
8,pazopanib,TMEM185B,COL9A3,DDX42,UPK1B,SCG2,LARP6,PRKCQ,SCN3A,CD74,...,KDM5D,NEBL,XBP1,ALDH1A1,CIAO3,GOLT1B,ALDH3A1,AKR1B10,ZFP36,RPS4Y1


In [23]:
# Filter bindingdb drugs present in l1000 data
selected_names = pd.Series(df_bindingdb['Name'].unique()).apply(lambda x: any(substring in x for substring in set(df_up['0'])))

# df_bindingdb2 = df_bindingdb[df_bindingdb['Name'].apply(lambda x: any(substring in x for substring in set(df_up['0'])))]
# df_bindingdb2


KeyboardInterrupt: 

In [ ]:
# One-hot encoding of drug SMILES
S = pd.Series(X_drugs.unique()).apply(utils.smiles2onehot)
S_dict = dict(zip(X_drugs.unique(), S))
df_drugs = [S_dict[i] for i in X_drugs]
one_hot_drugs = np.array(df_drugs)
print(f'One-hot encoding of drug: {one_hot_drugs.shape}')

In [ ]:
    # Extract down genes
df_down = df_l1000[df_l1000['0'].str.contains('down')]
# print(df_l1000_down.head())

# Clean the drug names in the replicates - down
df_down['0'] = df_down['0'].apply(ut.extract_drug_name)
df_down.head()

C:\Users\debna\AppData\Local\Temp\ipykernel_57400\3282555025.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_down['0'] = df_down['0'].apply(utils.extract_drug_name)


,0,2,3,4,5,6,7,8,9,10,...,242,243,244,245,246,247,248,249,250,251
1,afatinib,PCNA,S100A7,DNMT1,S100P,S100A9,PUF60,TMEM45A,GOLT1B,PRSS23,...,AKAP12,SCEL,CTSC,EAF2,MORF4L1,KCNK3,MYB,MAF,LTF,MFNG
3,erlotinib,PCP4,SPINK1,STEAP1,HOXC6,ITGB1BP1,MRPS16,XIST,UCHL1,FABP4,...,PEG3,WBP1L,SCG5,ATP5F1E,CCL19,EGLN1,MAST4,ATP6V1H,GPX2,EBP
5,neratinib,RPS4Y1,KRT18,THBS1,IGFBP3,PRSS23,ALDH1A3,CLU,PTCH1,CSGALNACT1,...,HIST1H2AC,PPM1H,ENPP2,ACP6,PEBP1,ATP1B3,COL14A1,SMAD3,RAB11FIP1,MANSC1
7,lapatinib,PUF60,HSPA1A,SPINK1,C3orf14,CYP1B1,MMP12,RRS1,KRT23,TSPYL5,...,MINPP1,PSMB10,STK10,PPP3CA,GAREM1,TTC19,EEF1D,PITRM1,CCN3,IL1R1
9,pazopanib,MRPS16,PIN1,XIST,LSM5,CXCL14,CDC20,F13A1,TTC39A,PRDX2,...,PPP1R2,NCF1C,MTCL1,RNF19B,MAOA,CRISPLD2,OSBPL8,SNX10,BBX,PPFIA1


In [9]:
# df_down.to_csv('../data/df_down.csv', index=False)

In [10]:
# drug_l1000_list = []
# for smiles, drug in smiles_cmap.items():
#     for drug_l1000 in df_down['0']:
#         if drug.lower() == drug_l1000.lower():
#             if drug not in drug_l1000_list:
#                 drug_l1000_list.append(drug)

# print(drug_l1000_list)

['L-theanine', 'L-citrulline', 'BRD-A18795974']


In [ ]:
landmark_genes = pd.read_csv('../data/landmark_genes.csv', header=None)

In [ ]:
data_reg_list = []
for drug in list(df_bindingdb2['Name']):
    drug_count = 0
    df_reg = landmark_genes
    df_reg['up'] = [0] * 978
    df_reg['down'] = [0] * 978
    for drug_name in df_down['0']:
        if drug_name == drug:
            drug_count += 1
    filtered_up = df_up[df_up['0'] == drug]
    filtered_down = df_down[df_down['0'] == drug]
    array_up = filtered_up.iloc[:, 1:].values
    array_up = array_up.flatten()
    array_down = filtered_down.iloc[:, 1:].values
    array_down = array_down.flatten()
    for item in array_up:
        df_reg.loc[df_reg[0] == item, 'up'] += 1
    for item in array_down:
        df_reg.loc[df_reg[0] == item, 'down'] += 1
    df_reg = df_reg.iloc[:, 1:] / drug_count
    df_reg = df_reg.values
    data_reg_list.append(df_reg)

data = np.stack(data_reg_list)
print(data.shape)

(3, 978, 2)
